In [ ]:
import tensorflow as tf
from tensorflow.keras import models
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False
import os
import numpy as np
from PIL import Image
import time
import functools
import tensorflow as tf
# import tensorflow.contrib.eager as tfe

from tensorflow.keras.preprocessing import image as kp_image


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

from matplotlib.dviread import Tfm

In [ ]:

# Paths to content and style images
content_path = "input_image.jpg"
style_path = "style.jpg"

In [ ]:
import tensorflow as tf
from tensorflow.keras import models

# Load the pre-trained model and weights
loaded_model = tf.keras.models.load_model('path_to_model.h5')

# Load the pre-trained weights (if saved separately)
loaded_model.load_weights('path_to_weights.h5')

# Use the loaded model to perform tasks such as style transfer
output = loaded_model(content_image, style_image)

# Display the stylized image
plt.imshow(stylized_image)
plt.axis('off')
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from PIL import Image
import numpy as np

# Content layer where we will pull our feature maps
content_layers = ['conv5_block1_1_bn'] 

# Style layers we are interested in
style_layers = ['conv5_block1_1_bn', 'conv2_block1_2_bn']  # Add more style layers if needed

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def get_model(style_layers, content_layers):
    """Creates a model that outputs intermediate layers of ResNet50.
    
    Args:
        style_layers: List of layer names for style representation.
        content_layers: List of layer names for content representation.
        
    Returns:
        A Keras model that takes image inputs and outputs the specified intermediate layers.
    """
    # Load the pre-trained ResNet50 model
    resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
    resnet.trainable = False
    
    # Get the output layers corresponding to style and content layers
    style_outputs = [resnet.get_layer(name).output for name in style_layers]
    content_outputs = [resnet.get_layer(name).output for name in content_layers]
    
    # Concatenate the outputs for both style and content layers
    model_outputs = style_outputs + content_outputs
    
    # Define the model with ResNet50 input and the selected intermediate layers as outputs
    model = models.Model(resnet.input, model_outputs)
    
    return model

def load_and_process_img(path):
    img = Image.open(path)
    img = np.array(img)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    img = tf.convert_to_tensor(img)
    img = tf.expand_dims(img, axis=0)
    return img

def deprocess_img(processed_img):
    x = processed_img.numpy()
    x = x.squeeze(0)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load the pre-trained model and weights
model = get_model(style_layers, content_layers)
model.summary()
model.load_weights('model_weights.weights.h5')
model.summary()

# Paths to content and style images
content_path = "input_image.jpg"
style_path = "style.jpg"

# Load and preprocess content and style images
content_image = load_and_process_img(content_path)
style_image = load_and_process_img(style_path)

# Forward pass through the model to get the feature maps for both content and style images
content_features = model(content_image)[-num_content_layers:]
style_features = model(style_image)[:num_style_layers]

# Concatenate the feature maps
concatenated_features = style_features + content_features

# Run the style transfer process on the concatenated feature maps
stylized_features = style_transfer_process(concatenated_features, max_iterations=5)

# Deprocess the stylized feature maps
stylized_image = deprocess_img(stylized_features)

# Display or save the stylized image as needed
plt.imshow(stylized_image)
plt.axis('off')
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from PIL import Image
import numpy as np

# Content layer where we will pull our feature maps
content_layers = ['conv5_block1_1_bn'] 

# Style layers we are interested in
style_layers = ['conv5_block1_1_bn', 'conv2_block1_2_bn']  # Add more style layers if needed

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def get_model(style_layers, content_layers):
    """Creates a model that outputs intermediate layers of ResNet50.
    
    Args:
        style_layers: List of layer names for style representation.
        content_layers: List of layer names for content representation.
        
    Returns:
        A Keras model that takes image inputs and outputs the specified intermediate layers.
    """
    # Load the pre-trained ResNet50 model
    resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
    resnet.trainable = False
    
    # Get the output layers corresponding to style and content layers
    style_outputs = [resnet.get_layer(name).output for name in style_layers]
    content_outputs = [resnet.get_layer(name).output for name in content_layers]
    
    # Concatenate the outputs for both style and content layers
    model_outputs = style_outputs + content_outputs
    
    # Define the model with ResNet50 input and the selected intermediate layers as outputs
    model = models.Model(resnet.input, model_outputs)
    
    return model

def load_and_process_img(path):
    img = Image.open(path)
    img = np.array(img)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    img = tf.convert_to_tensor(img)
    img = tf.expand_dims(img, axis=0)
    return img

def deprocess_img(processed_img):
    x = processed_img.numpy()
    x = x.squeeze(0)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load the pre-trained model and weights
model = get_model(style_layers, content_layers)
model.load_weights('model_weights.weights.h5')

# Paths to content and style images
content_path = "input_image.jpg"
style_path = "style.jpg"

# Load and preprocess content and style images
content_image = load_and_process_img(content_path)
style_image = load_and_process_img(style_path)

# Forward pass through the model to get the feature maps for both content and style images
content_features = model(content_image)[-num_content_layers:]
style_features = model(style_image)[:num_style_layers]

# Concatenate the feature maps
concatenated_features = style_features + content_features

# Run the style transfer process on the concatenated feature maps
stylized_features = style_transfer_process(concatenated_features, max_iterations=5)

# Deprocess the stylized feature maps
stylized_image = deprocess_img(stylized_features)

# Display or save the stylized image as needed
plt.imshow(stylized_image)
plt.axis('off')
plt.show()


In [1]:
import tensorflow as tf
from tensorflow.keras import models
from PIL import Image
import numpy as np

# Content layer where we will pull our feature maps
content_layers = ['conv5_block1_1_bn'] 

# Style layers we are interested in
style_layers = ['conv5_block1_1_bn', 'conv2_block1_2_bn']  # Add more style layers if needed

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def get_model(style_layers, content_layers):
    """Creates a model that outputs intermediate layers of ResNet50.
    
    Args:
        style_layers: List of layer names for style representation.
        content_layers: List of layer names for content representation.
        
    Returns:
        A Keras model that takes image inputs and outputs the specified intermediate layers.
    """
    # Load the pre-trained ResNet50 model
    resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
    resnet.trainable = False
    
    # Get the output layers corresponding to style and content layers
    style_outputs = [resnet.get_layer(name).output for name in style_layers]
    content_outputs = [resnet.get_layer(name).output for name in content_layers]
    
    # Concatenate the outputs for both style and content layers
    model_outputs = style_outputs + content_outputs
    
    # Define the model with ResNet50 input and the selected intermediate layers as outputs
    model = models.Model(resnet.input, model_outputs)
    
    return model

def load_and_process_img(path):
    img = Image.open(path)
    img = np.array(img)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    img = tf.convert_to_tensor(img)
    img = tf.expand_dims(img, axis=0)
    return img

def deprocess_img(processed_img):
    x = processed_img.numpy()
    x = x.squeeze(0)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load the pre-trained model and weights
model = get_model(style_layers, content_layers)
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 9,116,032 (34.77 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,116,032 (34.77 MB)

In [2]:
model.load_weights('model_weights.weights.h5')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 9,116,032 (34.77 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,116,032 (34.77 MB)

In [ ]:

# Paths to content and style images
content_path = "input_image.jpg"
style_path = "style.jpg"

# Load and preprocess content and style images
content_image = load_and_process_img(content_path)
style_image = load_and_process_img(style_path)

# Forward pass through the model to get the feature maps for both content and style images
content_features = model(content_image)[-num_content_layers:]
style_features = model(style_image)[:num_style_layers]

# Concatenate the feature maps
concatenated_features = style_features + content_features

# Run the style transfer process on the concatenated feature maps
stylized_features = style_transfer_process(concatenated_features, max_iterations=5)

# Deprocess the stylized feature maps
stylized_image = deprocess_img(stylized_features)

# Display or save the stylized image as needed
plt.imshow(stylized_image)
plt.axis('off')
plt.show()
